# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [4]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,saint-pierre,-21.3393,55.4781,24.82,73,20,0.00,RE,1701019405
1,1,albany,42.6001,-73.9662,5.65,58,96,1.34,US,1701018983
2,2,ilulissat,69.2167,-51.1000,0.01,86,75,1.03,GL,1701019333
3,3,badger,64.8000,-147.5333,-10.86,89,40,0.00,US,1701019252
4,4,grytviken,-54.2811,-36.5092,2.79,99,100,3.13,GS,1701019275


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
# Configure the map plot
# YOUR CODE HERE
# Configure the map scatter plot using hvplot
# Configure the map scatter plot using hvplot
import colorcet as cc  # Import a color map library, you can use any other color map library

# Configure the map scatter plot using hvplot with different colors for each city
map_plot = city_data_df.hvplot.scatter(
    x='Lng',
    y='Lat',
    c='City',  # Use 'City' column for color mapping
    cmap=cc.glasbey_light,  # Use a color map, you can choose a different one
    size='Humidity',
    title='City Map with Humidity',
    xlabel='Longitude',
    ylabel='Latitude',
    hover_cols=['City', 'Country', 'Humidity'],
    frame_width=600,
    frame_height=400,
    xlim=(city_data_df['Lng'].min(), city_data_df['Lng'].max()),
    ylim=(city_data_df['Lat'].min(), city_data_df['Lat'].max())
)

# Display the map
map_plot


:Scatter   [Lng]   (Lat,City,Humidity,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
# Narrow down cities that fit criteria
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] < 25) &  # Adjust temperature criteria as needed
    (city_data_df['Humidity'] < 70) &  # Adjust humidity criteria as needed
    (city_data_df['Cloudiness'] == 0) &  # Adjust cloudiness criteria as needed
    (city_data_df['Wind Speed'] < 5)  # Adjust wind speed criteria as needed
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
128,128,arar,30.9753,41.0381,19.01,37,0,3.09,SA,1701019436
144,144,ghat,24.9647,10.1728,20.28,23,0,2.69,LY,1701019332
146,146,nola,40.9220,14.5329,9.40,63,0,0.73,IT,1701019439
226,226,pacific grove,36.6177,-121.9166,13.06,54,0,2.06,US,1701019469
232,232,astaneh-ye ashrafiyeh,37.2596,49.9444,24.96,29,0,4.55,IR,1701019471


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
128,arar,SA,30.9753,41.0381,37,
144,ghat,LY,24.9647,10.1728,23,
146,nola,IT,40.9220,14.5329,63,
226,pacific grove,US,36.6177,-121.9166,54,
232,astaneh-ye ashrafiyeh,IR,37.2596,49.9444,29,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()


Starting hotel search
arar - nearest hotel: دار أوتيل Dar Otel
ghat - nearest hotel: فندق تاسيلي
nola - nearest hotel: Pamaran
pacific grove - nearest hotel: Pacific Grove Inn
astaneh-ye ashrafiyeh - nearest hotel: هتل دهدار
kangding - nearest hotel: 溜溜城格桑花大酒店
nagqu - nearest hotel: Naqu Hotel
metlili chaamba - nearest hotel: No hotel found
ha'il - nearest hotel: فندق كلاسيك
iranshahr - nearest hotel: هتل قصِر
barda - nearest hotel: Aida's Home
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
tindouf - nearest hotel: محمد بوسبي
page - nearest hotel: Clarion Inn Page - Lake Powell


,City,Country,Lat,Lng,Humidity,Hotel Name
128,arar,SA,30.9753,41.0381,37,دار أوتيل Dar Otel
144,ghat,LY,24.9647,10.1728,23,فندق تاسيلي
146,nola,IT,40.9220,14.5329,63,Pamaran
226,pacific grove,US,36.6177,-121.9166,54,Pacific Grove Inn
232,astaneh-ye ashrafiyeh,IR,37.2596,49.9444,29,هتل دهدار


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
%%capture --no-display
import colorcet as cc
import holoviews as hv

# Configure the map plot
# Use the 'Hotel Name' and 'Country' columns for additional information in the hover message
hotel_map = hotel_df.hvplot.scatter(
    x='Lng',
    y='Lat',
    c='Country',  # Use the 'Country' column for color
    cmap=cc.fire,  # Use the 'fire' color map from colorcet
    size='Humidity',
    title='City Map with Hotel Information',
    xlabel='Longitude',
    ylabel='Latitude',
    hover_cols=['City', 'Country', 'Humidity', 'Hotel Name'],
    frame_width=600,
    frame_height=400,
    xlim=(hotel_df['Lng'].min(), hotel_df['Lng'].max()),  # Set the x-axis limits
    ylim=(hotel_df['Lat'].min(), hotel_df['Lat'].max()),  # Set the y-axis limits
)

# Display the map
hotel_map


:Scatter   [Lng]   (Lat,Country,Humidity,City,Hotel Name)